# Alkane Simulation with OpenMM


In [ ]:
# OpenMM imports
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit

## Simulation Parameters

In [ ]:
# Set up our starting files to get initial configuration
import os

pdb_file = 
ff_file = 

pdb = app.PDBFile(pdb_file)
forcefield = app.ForceField(ff_file)

# Create OpenMM representation of the system
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [ ]:
# Set up the integrator

# System temperature
temperature =  # temperature in Kelvin

# Set temperature, coupling constant, and time step
integrator = mm.LangevinIntegrator()

integrator.setConstraintTolerance(1e-5)

In [ ]:
# Create simulation
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

# Minimization

The minimization will move the atoms slightly to get them in a more energetically favorable position.

In [ ]:
print('Minimizing...')

st_initial = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st_initial.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st_minimized = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st_minimized.getPotentialEnergy()}")

# Equilibration

We'll run the simulation for a few time steps to make sure the system is equilibrated. We usually do this at the start of a simulation to make sure the data we collect later isn't influenced by the starting position of the atoms.

In [ ]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator='\t'))

# Set initial velocities
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)

simulation.step(2500)

# Production

This is the portion of the simulation we will collect data from and analyze.

In [ ]:
import time

print('Running Production...')

# Begin timer
tinit=

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. 
# We do this because we want different information printed 
# from the production run than the equilibration run.
# Output basic simulation information below every 250000 steps (How many picoseconds is?) 

simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps 
simulation.reporters.append(app.DCDReporter(F'ethane_sim_{temperature}.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step()

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

## Visualization of Trajectory

The files we've output have the type dcd. We need a special program to view the coordinates. We are using a Python library called `mdtraj`

If you don't have `mdtraj`, you can install it

```
conda install -c conda-forge mdtraj  
conda install -c conda-forge nglview
```

In [ ]:
import mdtraj as md

traj = md.load(F'ethane_sim_{temperature}.dcd', top=pdb_file)

Next, we use a library called `NGLView` to watch our trajectory.

In [ ]:
import nglview as ngl

# Create a view (like creating a plot)

view = ngl.NGLWidget()
view.add_trajectory(traj)
view

## Analysis

We can calculate bond and angle distributions using `mdtraj`, and graph them using `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
bond_indices = [0, 4] # atoms to define the bond length

# Use mdtraj library to calculate bond length.
bond_length = md.compute_distances(traj, [bond_indices])
print(bond_length)

In [ ]:
# Create a figure plotting the bond length


It will be more informative to look at this information as a histogram.

In [ ]:
# Create histogram here.

# Analyzing Torsions

In [ ]:
phi_indices =  # atoms to define the torsion angle
phi = md.compute_dihedrals(traj, [phi_indices])

print(phi)

In [ ]:
plt.figure()


# Exercise 

Try changing the temperature and seeing how that affects simulation results.